In [1]:

import pandas as pd


In [7]:
!pip install pickle5
!pip install numpy pandas matplotlib seaborn scikit-learn transformers datasets  
!pip install torch torchvision torchaudio

  Using cached matplotlib-3.7.1-cp38-cp38-macosx_11_0_arm64.whl (7.3 MB)
  Using cached seaborn-0.12.2-py3-none-any.whl (293 kB)
  Using cached scikit_learn-1.2.2-cp38-cp38-macosx_12_0_arm64.whl (8.3 MB)
  Using cached transformers-4.28.1-py3-none-any.whl (7.0 MB)
  Using cached datasets-2.11.0-py3-none-any.whl (468 kB)
  Using cached contourpy-1.0.7-cp38-cp38-macosx_11_0_arm64.whl (229 kB)
  Using cached Pillow-9.5.0-cp38-cp38-macosx_11_0_arm64.whl (3.1 MB)
  Using cached fonttools-4.39.3-py3-none-any.whl (1.0 MB)
  Using cached cycler-0.11.0-py3-none-any.whl (6.4 kB)
  Using cached kiwisolver-1.4.4-cp38-cp38-macosx_11_0_arm64.whl (63 kB)
  Using cached threadpoolctl-3.1.0-py3-none-any.whl (14 kB)
  Using cached joblib-1.2.0-py3-none-any.whl (297 kB)
  Using cached scipy-1.10.1-cp38-cp38-macosx_12_0_arm64.whl (28.8 MB)
  Using cached tqdm-4.65.0-py3-none-any.whl (77 kB)
  Using cached tokenizers-0.13.3-cp38-cp38-macosx_12_0_arm64.whl (3.9 MB)
  Using cached huggingface_hub-0.13.4-py3-

In [3]:
data = pd.read_csv("/Users/vikashmediboina/Downloads/early-detection-of-3d-printing-issues/train.csv")

In [4]:
data["img_path"]= ["/Users/vikashmediboina/Downloads/early-detection-of-3d-printing-issues/images/" + each for each in data["img_path"]]

In [5]:
data.columns

Index(['img_path', 'printer_id', 'print_id', 'has_under_extrusion'], dtype='object')

In [8]:
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
from PIL import Image
import os
import random
from transformers import ViTForImageClassification, ViTFeatureExtractor, ViTImageProcessor, ViTModel

In [9]:


# Define the image transforms for data augmentation
image_transforms = transforms.Compose([
#     transforms.ToPILImage(),
    transforms.RandomResizedCrop(size=224, scale=(0.95, 1.0)),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])

# Define a custom dataset class to load images and their labels
class CustomImageDatasets(Dataset):
    def __init__(self, data, transform=None):
        self.dataset_path = data
        self.transform = transform
        self.images = []
        self.labels = []
        for index, row in data.iterrows():
                        self.images.append(row['img_path'])
                        self.labels.append(row['has_under_extrusion'])

    def __len__(self):
        return len(self.images)

    def __getitem__(self, idx):
        image_path = self.images[idx]
        label = self.labels[idx]
        image = Image.open(image_path).convert("RGB")
        if self.transform:
            image = self.transform(image)
        return image, label

# Define the dataset
dataset = CustomImageDatasets(data, transform=image_transforms)

# Split the dataset into training and testing datasets
split_ratio = 0.8
split_idx = int(len(dataset) * split_ratio)
indices = list(range(len(dataset)))
random.shuffle(indices)
train_indices = indices[:split_idx]
test_indices = indices[split_idx:]
train_dataset = torch.utils.data.Subset(dataset, train_indices)
test_dataset = torch.utils.data.Subset(dataset, test_indices)


In [10]:
batch_size = 16
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=4)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False, num_workers=4)
# train_loader.images


In [11]:
train_loader

In [13]:
for images,labels in train_loader:
    print(images.shape)   # Should print something like torch.Size([32, 3, 224, 224])
    break

Traceback (most recent call last):
  File "<string>", line 1, in <module>
  File "/Users/vikashmediboina/miniforge3/envs/mlp/lib/python3.8/multiprocessing/spawn.py", line 116, in spawn_main
    exitcode = _main(fd, parent_sentinel)
  File "/Users/vikashmediboina/miniforge3/envs/mlp/lib/python3.8/multiprocessing/spawn.py", line 126, in _main
    self = reduction.pickle.load(from_parent)
AttributeError: Can't get attribute 'CustomImageDatasets' on <module '__main__' (built-in)>


KeyboardInterrupt: 

In [189]:
# Define the ViT model and feature extractor
model = ViTForImageClassification.from_pretrained('google/vit-base-patch16-224')
img_processor = ViTImageProcessor.from_pretrained('google/vit-base-patch16-224-in21k')

# Define the loss function and optimizer
loss_function = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-5)

# Define the device to train the model on
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)

# Define the training loop
num_epochs = 10
for epoch in range(num_epochs):
    model.train()
    train_loss = 0.0
    train_accuracy = 0.0
    image_list = []
    for images, labels in train_loader:
        image_list.extend([image for image in images])
        labels = labels.to(device)
        processed_imgs = img_processor(image_list, return_tensors='pt', data_format='channels_first').to(device)
        outputs = model(**processed_imgs)
        loss = loss_function(outputs.logits, labels)
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()
        image_list = []
        print("loss ->", loss)
    for images, labels in test_loader:
        image_list.extend([image for image in images])
        labels = labels.to(device)
        processed_imgs = img_processor(image_list, return_tensors='pt', data_format='channels_first').to(device)
        outputs = model(**processed_imgs)
        val_loss = loss_function(outputs.logits, labels)
        image_list = []
        print("Validation loss ->", val_loss)

loss -> tensor(9.6779, grad_fn=<NllLossBackward0>)
loss -> tensor(8.3220, grad_fn=<NllLossBackward0>)
loss -> tensor(8.6809, grad_fn=<NllLossBackward0>)
loss -> tensor(7.9111, grad_fn=<NllLossBackward0>)
loss -> tensor(7.4423, grad_fn=<NllLossBackward0>)
loss -> tensor(7.4664, grad_fn=<NllLossBackward0>)
loss -> tensor(6.5214, grad_fn=<NllLossBackward0>)
loss -> tensor(6.0334, grad_fn=<NllLossBackward0>)
loss -> tensor(5.9971, grad_fn=<NllLossBackward0>)
loss -> tensor(5.2286, grad_fn=<NllLossBackward0>)
loss -> tensor(4.7384, grad_fn=<NllLossBackward0>)
loss -> tensor(4.6105, grad_fn=<NllLossBackward0>)
loss -> tensor(4.5925, grad_fn=<NllLossBackward0>)


KeyboardInterrupt: 

In [13]:
from sklearn.metrics import f1_score, accuracy_score

# Define the ViT model and feature extractor
model = ViTForImageClassification.from_pretrained('google/vit-base-patch16-224')
img_processor = ViTImageProcessor.from_pretrained('google/vit-base-patch16-224-in21k')

# Define the loss function and optimizer
loss_function = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-5)

# Define the device to train the model on
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)

# Define the training loop
num_epochs = 10
for epoch in range(num_epochs):
    model.train()
    train_loss = 0.0
    train_accuracy = 0.0
    train_f1 = 0.0
    image_list = []
    for images, labels in train_loader:
        image_list.extend([image for image in images])
        labels = labels.to(device)
        processed_imgs = img_processor(image_list, return_tensors='pt', data_format='channels_first').to(device)
        outputs = model(**processed_imgs)
        loss = loss_function(outputs.logits, labels)
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()
        image_list = []
        train_loss += loss.item()
        _, preds = torch.max(outputs.logits, dim=1)
        train_accuracy += accuracy_score(labels.cpu(), preds.cpu())
        train_f1 += f1_score(labels.cpu(), preds.cpu(), average='macro')
        
    train_loss /= len(train_loader)
    train_accuracy /= len(train_loader)
    train_f1 /= len(train_loader)
    
    model.eval()
    val_loss = 0.0
    val_accuracy = 0.0
    val_f1 = 0.0
    with torch.no_grad():
        for images, labels in test_loader:
            image_list.extend([image for image in images])
            labels = labels.to(device)
            processed_imgs = img_processor(image_list, return_tensors='pt', data_format='channels_first').to(device)
            outputs = model(**processed_imgs)
            loss = loss_function(outputs.logits, labels)
            image_list = []
            val_loss += loss.item()
            _, preds = torch.max(outputs.logits, dim=1)
            val_accuracy += accuracy_score(labels.cpu(), preds.cpu())
            val_f1 += f1_score(labels.cpu(), preds.cpu(), average='macro')
    
    val_loss /= len(test_loader)
    val_accuracy /= len(test_loader)
    val_f1 /= len(test_loader)
    
    print(f"Epoch {epoch+1} | Train loss: {train_loss:.4f} | Train accuracy: {train_accuracy:.4f} | Train F1-score: {train_f1:.4f} | Validation loss: {val_loss:.4f} | Validation accuracy: {val_accuracy:.4f} | Validation F1-score: {val_f1:.4f}")


/opt/conda/lib/python3.7/site-packages/torch/utils/data/dataloader.py:557: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))
/opt/conda/lib/python3.7/site-packages/torch/utils/data/dataloader.py:557: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


Epoch 1 | Train loss: 0.0814 | Train accuracy: 0.9758 | Train F1-score: 0.9734 | Validation loss: 0.0080 | Validation accuracy: 0.9971 | Validation F1-score: 0.9968


/opt/conda/lib/python3.7/site-packages/torch/utils/data/dataloader.py:557: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))
/opt/conda/lib/python3.7/site-packages/torch/utils/data/dataloader.py:557: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


Epoch 2 | Train loss: 0.0057 | Train accuracy: 0.9982 | Train F1-score: 0.9981 | Validation loss: 0.0185 | Validation accuracy: 0.9940 | Validation F1-score: 0.9936


/opt/conda/lib/python3.7/site-packages/torch/utils/data/dataloader.py:557: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))
/opt/conda/lib/python3.7/site-packages/torch/utils/data/dataloader.py:557: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


...

Epoch 8 | Train loss: 0.0009 | Train accuracy: 0.9997 | Train F1-score: 0.9997 | Validation loss: 0.0017 | Validation accuracy: 0.9993 | Validation F1-score: 0.9992


/opt/conda/lib/python3.7/site-packages/torch/utils/data/dataloader.py:557: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))
/opt/conda/lib/python3.7/site-packages/torch/utils/data/dataloader.py:557: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


Epoch 9 | Train loss: 0.0010 | Train accuracy: 0.9997 | Train F1-score: 0.9997 | Validation loss: 0.0019 | Validation accuracy: 0.9994 | Validation F1-score: 0.9994


/opt/conda/lib/python3.7/site-packages/torch/utils/data/dataloader.py:557: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))
/opt/conda/lib/python3.7/site-packages/torch/utils/data/dataloader.py:557: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


Epoch 10 | Train loss: 0.0009 | Train accuracy: 0.9998 | Train F1-score: 0.9998 | Validation loss: 0.0005 | Validation accuracy: 0.9998 | Validation F1-score: 0.9998


In [16]:
torch.save(model.state_dict(), "/kaggle/working/Vision")

In [19]:
trained_model = ViTForImageClassification.from_pretrained('google/vit-base-patch16-224')
trained_model.load_state_dict(torch.load("/kaggle/working/Vision"))
trained_model.eval()

ViTForImageClassification(
  (vit): ViTModel(
    (embeddings): ViTEmbeddings(
      (patch_embeddings): ViTPatchEmbeddings(
        (projection): Conv2d(3, 768, kernel_size=(16, 16), stride=(16, 16))
      )
      (dropout): Dropout(p=0.0, inplace=False)
    )
    (encoder): ViTEncoder(
      (layer): ModuleList(
        (0): ViTLayer(
          (attention): ViTAttention(
            (attention): ViTSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.0, inplace=False)
            )
            (output): ViTSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.0, inplace=False)
            )
          )
          (intermediate): ViTIntermediate(
            (dense): Linear(in_features=768, out_

In [20]:
test_data = pd.read_csv("/kaggle/input/early-detection-of-3d-printing-issues/test.csv")
test_data["img_path"]= ["/kaggle/input/early-detection-of-3d-printing-issues/images/" + each for each in test_data["img_path"]]

In [25]:
test_data.head()

,img_path,printer_id,print_id
0,/kaggle/input/early-detection-of-3d-printing-i...,101,1678578332
1,/kaggle/input/early-detection-of-3d-printing-i...,101,1678578332
2,/kaggle/input/early-detection-of-3d-printing-i...,101,1678578332
3,/kaggle/input/early-detection-of-3d-printing-i...,101,1678578332
4,/kaggle/input/early-detection-of-3d-printing-i...,101,1678578332


In [30]:
from sklearn.metrics import f1_score, accuracy_score

# Define the ViT model and feature extractor
img_processor = ViTImageProcessor.from_pretrained('google/vit-base-patch16-224-in21k')

# Define the device to train the model on
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
trained_model = trained_model.to(device)

predicted_labels=[] 
submission = {}
# Define the testing loop
with torch.no_grad():
    for image_path in test_data["img_path"]:
        image = Image.open(image_path).convert("RGB")
        processed_img = img_processor(image, return_tensors='pt', data_format='channels_first').to(device)
        outputs = trained_model(**processed_img)
        _, preds = torch.max(outputs.logits, dim=1)
        print(preds.item())
        predicted_labels.append(preds)
        submission[image_path] = preds
        
    
        



1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1


In [74]:
# 'img_path', 'printer_id', 'print_id', 'has_under_extrusion
# Convert dictionary to DataFrame
df = pd.DataFrame(list(submission.items()), columns=['img_path', 'has_under_extrusion'])

df.columns()

TypeError: 'Index' object is not callable

In [73]:

df['has_under_extrusion'] = df.apply( lambda x: x['has_under_extrusion'].item())

KeyError: 'has_under_extrusion'

In [66]:
# len('/kaggle/input/early-detection-of-3d-printing-issues/images/')
df['img_path']=df['img_path'].str.slice(start=59)

In [67]:
df.head()

,img_path,has_under_extrusion
0,101/1678578332/1678578538.704966.jpg,1
1,101/1678578332/1678578539.108019.jpg,1
2,101/1678578332/1678578539.512872.jpg,1
3,101/1678578332/1678578539.916711.jpg,1
4,101/1678578332/1678578540.329089.jpg,1


In [68]:

# # Write DataFrame to CSV file
df.to_csv('output.csv', index=False)

In [13]:

import torch
from torch import nn
import torch.nn.functional as F

class MLP(nn.Module):
    def __init__(self, in_channels, num_classes, hidden_sizes=[128, 64], dropout_probability=[0.5,0.7]):
        super(MLP, self).__init__()
        assert len(hidden_sizes) >= 1 , "specify at least one hidden layer"
        
        self.layers = self.create_layers(in_channels, num_classes, hidden_sizes, dropout_probability)


    def create_layers(self, in_channels, num_classes, hidden_sizes, dropout_probability):
        layers = []
        layer_sizes = [in_channels] + hidden_sizes + [num_classes]
        for i in range(len(layer_sizes)-1):
            layers.append(nn.Linear(layer_sizes[i], layer_sizes[i+1]))
            if i < len(layer_sizes)-2:
                layers.append(nn.BatchNorm1d(layer_sizes[i+1]))
                layers.append(nn.ReLU())
                layers.append(nn.Dropout(dropout_probability[i]))
            else:
                layers.append(nn.Softmax(dim=1))
        return nn.Sequential(*layers)

    def forward(self, x):
        out = x.view(x.shape[0], -1)
        out = self.layers(out)
        return out

In [34]:

class CombinedModel(nn.Module):
    def __init__(self, modality1, configuration,device):
        super().__init__()
        self.device = device
        self.modality1 = modality1.to(self.device)
        self.config = configuration

        self.head = MLP(in_channels=self._calculate_in_features(),
                            num_classes=self.config['Models']['mlp_num_classes'],
                            hidden_sizes=self.config['Models']['mlp_hidden_sizes'], 
                            dropout_probability= self.config['Models']['mlp_dropout_prob']).to(self.device)

        if(configuration['Models']['encoder_finetuning'] == False):
            for param in self.modality1.parameters():
                param.requires_grad = False

  

        for param in self.head.parameters():
            param.requires_grad = True

    def forward(self, input1):
        image_output = self.modality1(input1)['last_hidden_state'].to(self.device)
        image_output = torch.nn.Flatten()(image_output).to(self.device)
        head_output = self.head(image_output).to(self.device)
        return head_output

    def _calculate_in_features(self):
        # Create an example input and pass it through the network to get the output size
        image_list=[]
        img_batch = torch.randint(0, 255, size=(self.config['Dataset']['batch_size'], 3, 224, 224)).float()
        image_list.extend([image for image in img_batch])
        img_processor = self.config['Models']['image_processor']
        input1 = img_processor(image_list, return_tensors='pt').to(self.device) 
        image_output = self.modality1(**input1)['last_hidden_state'].to(self.device)
        image_output = torch.nn.Flatten()(image_output).to(self.device)
        return image_output.shape[1]


In [35]:
model_vit = ViTModel.from_pretrained('google/vit-base-patch16-224')
img_processor = ViTImageProcessor.from_pretrained('google/vit-base-patch16-224-in21k')

Some weights of the model checkpoint at google/vit-base-patch16-224 were not used when initializing ViTModel: ['classifier.weight', 'classifier.bias']
- This IS expected if you are initializing ViTModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing ViTModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [36]:
configuration={}
configuration['Models']={}
configuration['Models']['mlp_num_classes']=2
configuration['Models']['mlp_hidden_sizes']= [1024,512,256]
configuration['Models']['mlp_dropout_prob']=[0.5,0.5,0.4]
configuration['Models']['encoder_finetuning']=True
configuration['Models']['image_processor']=img_processor
configuration['Dataset']={}
configuration['Dataset']['batch_size']=16


In [37]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model=CombinedModel(model_vit,configuration,device)

In [38]:
from sklearn.metrics import f1_score, accuracy_score
from tqdm import tqdm


# Define the loss function and optimizer
loss_function = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-5)


# Define the training loop
num_epochs = 10
for epoch in tqdm(range(num_epochs)):
    model.train()
    train_loss = 0.0
    train_accuracy = 0.0
    train_f1 = 0.0
    image_list = []
    for images, labels in train_loader:
        image_list.extend([image for image in images])
        labels = labels.to(device)
        processed_imgs = img_processor(image_list, return_tensors='pt', data_format='channels_first').to(device)
        outputs = model(processed_imgs['pixel_values'])
        loss = loss_function(outputs, labels)
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()
        image_list = []
        train_loss += loss.item()
        _, preds = torch.max(outputs, dim=1)
        train_accuracy += accuracy_score(labels.cpu(), preds.cpu())
        train_f1 += f1_score(labels.cpu(), preds.cpu(), average='macro')
        
    train_loss /= len(train_loader)
    train_accuracy /= len(train_loader)
    train_f1 /= len(train_loader)
    
    model.eval()
    val_loss = 0.0
    val_accuracy = 0.0
    val_f1 = 0.0
    with torch.no_grad():
        for images, labels in test_loader:
            image_list.extend([image for image in images])
            labels = labels.to(device)
            processed_imgs = img_processor(image_list, return_tensors='pt', data_format='channels_first').to(device)
            outputs = model(processed_imgs['pixel_values'])
            loss = loss_function(outputs, labels)
            image_list = []
            val_loss += loss.item()
            _, preds = torch.max(outputs, dim=1)
            val_accuracy += accuracy_score(labels.cpu(), preds.cpu())
            val_f1 += f1_score(labels.cpu(), preds.cpu(), average='macro')
    
    val_loss /= len(test_loader)
    val_accuracy /= len(test_loader)
    val_f1 /= len(test_loader)
    
    print(f"Epoch {epoch+1} | Train loss: {train_loss:.4f} | Train accuracy: {train_accuracy:.4f} | Train F1-score: {train_f1:.4f} | Validation loss: {val_loss:.4f} | Validation accuracy: {val_accuracy:.4f} | Validation F1-score: {val_f1:.4f}")
torch.save(model.state_dict(), "/kaggle/working/Vision")

  0%|          | 0/10 [00:00<?, ?it/s]Traceback (most recent call last):
  File "<string>", line 1, in <module>
  File "/Users/vikashmediboina/mambaforge/envs/printing/lib/python3.10/multiprocessing/spawn.py", line 116, in spawn_main
    exitcode = _main(fd, parent_sentinel)
  File "/Users/vikashmediboina/mambaforge/envs/printing/lib/python3.10/multiprocessing/spawn.py", line 126, in _main
    self = reduction.pickle.load(from_parent)
AttributeError: Can't get attribute 'CustomImageDataset' on <module '__main__' (built-in)>
  0%|          | 0/10 [00:44<?, ?it/s]


KeyboardInterrupt: 